In [3]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.MiscUtils import *
from Utils.ImageUtils import *
from Utils.MathUtils import *
import scipy.optimize 
square_side = 12.5
# %load_ext autoreload
# %autoreload 2


In [8]:
def run(mode):

    if (mode == "ir"):
        folder_name = r'C:\Users\Asus\Documents\Shuna Ni\images\ir_mod'   #folder where IR images are stored
        save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\ir_saved"  #folder where IR images are to be saved
    elif (mode=="rgb"):
        folder_name = r'C:\Users\Asus\Documents\Shuna Ni\images\rgb_mod'    #folder where RGB images are stored
        save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\rgb_saved"  #folder where IR images are to be saved

    images = loadImages(folder_name)
    h, w = [6,7]
    all_image_corners = getImagesPoints(images, h, w)
    print(len(all_image_corners) , len(all_image_corners[0]))
    world_corners = getWorldPoints(square_side, h, w)

    displayCorners(images, all_image_corners, h, w, save_folder)

    print("Calculating H for %d images", len(images))
    all_H_init = getAllH(all_image_corners, square_side, h, w)
    print("Calculating B")
    B_init = getB(all_H_init)
    print("Estimated B = ", B_init)
    print("Calculating A")
    A_init = getA(B_init)
    print("Initialized A = ",A_init)
    print("Calculating rotation and translation")
    all_RT_init = getRotationAndTrans(A_init, all_H_init)
    print("Init Kc")
    kc_init = np.array([0,0]).reshape(2,1)
    print("Initialized kc = ", kc_init)


    return A_init, kc_init, all_RT_init, all_image_corners, all_H_init, world_corners

In [9]:
#run once

rgb_A, rgb_kc, rgb_rt, rgb_corners, rgb_h, wc = run("rgb")
ir_A, ir_kc, ir_rt, ir_corners, ir_h, wc_1 = run("ir")

Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\rgb_mod
7
7 42
Calculating H for %d images 7
Calculating B
B matrix is  [ 2.39542221e-06 -2.24840500e-08  2.42254636e-06 -7.56087644e-04
 -6.04368049e-04  9.99999532e-01]
Estimated B =  [[ 2.39542221e-06 -2.24840500e-08 -7.56087644e-04]
 [-2.24840500e-08  2.42254636e-06 -6.04368049e-04]
 [-7.56087644e-04 -6.04368049e-04  9.99999532e-01]]
Calculating A
Initialized A =  [[503.3878367    4.69860669 318.00792333]
 [  0.         500.58360707 252.42784461]
 [  0.           0.           1.        ]]
Calculating rotation and translation
Init Kc
Initialized kc =  [[0]
 [0]]
Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\ir_mod
7
7 42
Calculating H for %d images 7
Calculating B
B matrix is  [ 1.08070634e-06  4.91639657e-08  1.16190843e-06 -4.02615985e-04
 -4.41719989e-04  9.99999821e-01]
Estimated B =  [[ 1.08070634e-06  4.91639657e-08 -4.02615985e-04]
 [ 4.91639657e-08  1.16190843e-06 -4.41719989e-04]
 [-4.02615985e-04 -

In [10]:
# reprojects control points seen by the RGB camera onto the IR camera's frame of reference and finds distance 


rgb_to_ir = []
#calculates the homogenous transformation matrix
for i in range(len(rgb_h)):
    H_rgb_inv = np.linalg.inv(rgb_h[i])
    H_ir = ir_h[i]
    rgb_to_ir.append(np.matmul(H_ir, H_rgb_inv))

# print(H_inv.shape, rgb_corners[0][0])
error_by_image = []
all_projections = []
for i in range(len(rgb_corners)):
    projections = []
    error = 0
    for j in range(len(rgb_corners[i])):
        rgb_corner = rgb_corners[i][j]
        corner2 = np.array([rgb_corner[0],rgb_corner[1],1]).reshape(3,1)
        calculated_point_3 = np.matmul(rgb_to_ir[i],corner2)
        calculated_point_2 = np.array([calculated_point_3[0][0]/calculated_point_3[2][0],calculated_point_3[1][0]/calculated_point_3[2][0] ])
        projections.append(calculated_point_2)
        actual_detected_corner = ir_corners[i][j]
        
        error += np.linalg.norm(calculated_point_2 - actual_detected_corner)
    all_projections.append(projections)
    mean_error = error/j
    error_by_image.append(mean_error) 

print("Re-projection error for each image", error_by_image)
print("Mean error", sum(error_by_image)/len(error_by_image))

Re-projection error for each image [0.697342344273477, 0.5814622574160199, 0.9821334309978444, 1.0366495552007313, 0.8302213141083916, 0.6632218343292802, 0.7428255508159852]
Mean error 0.7905508981631043


In [11]:
open_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\ir_mod" #Enter folder where IR images are stored
save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\reproj_ir" #Enter folder where reprojected point images are to be stored
images = loadImages(open_folder)
for i,image in enumerate(images):
    drawn = cv2.drawChessboardCorners(image, (7, 6), ir_corners[i], True)
    # cv2.imshow('Chessboard',drawn)
    # cv2.waitKey(0)
    for point in all_projections[i]:
        x = int(point[0])
        y = int(point[1])
        image = cv2.circle(drawn, (x, y), 5, (0, 0, 255), 1)
        
    filename = save_folder + "\\" + str(i) + "reproj.png"
    cv2.imwrite(filename, image)

Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\ir_mod
